# Import

In [1]:
import sys
import torch
import torch.nn as nn
from Server import Server
from Client import Client
from Individual import Individual
from shakespeare_model import CharLSTM
from statistics import mean
import tkinter as tk
from tkinter import filedialog
import json

# Parameters

In [2]:
# Constants for FL training
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(DEVICE)
FRACTION_CLIENTS = 0.1  # Fraction of clients selected per round (C)
BATCH_SIZE = 100 # Batch size for local training
MOMENTUM = 0  # Momentum for SGD optimizer
LOG_FREQUENCY = 10 # Frequency of logging training progress

cpu


## Utility functions

In [3]:
"""
Utility function used both in the centralized and federated learning
Computes the accuracy and the loss on the validation/test set depending on the dataloader passed
"""
def evaluate(model, dataloader, criterion, DEVICE):
    model.eval()  # Set the model to evaluation mode
    running_corrects = 0
    total_samples = 0  # Total samples counter
    losses = []

    with torch.no_grad():
        for data, targets in dataloader:
            data = data.to(DEVICE)
            targets = targets.to(DEVICE)
            hidden = model.init_hidden(data.size(0))
            hidden = (hidden[0].to(DEVICE), hidden[1].to(DEVICE))
            outputs, _ = model(data, hidden)
            outputs_flat = outputs.view(-1, model.vocab_size)
            targets_flat = targets.view(-1)

            loss = criterion(outputs_flat, targets_flat)
            losses.append(loss.item())

            _, preds = outputs_flat.max(1)
            #running_corrects += torch.sum(preds == targets_flat).item()
            running_corrects += (preds == targets_flat).sum().item()
            total_samples += targets_flat.size(0)

    accuracy = (running_corrects / total_samples) * 100
    return accuracy, sum(losses) / len(losses)


def test(global_model, test_loader, criterion, DEVICE):
    """
    Evaluate the global model on the test dataset.

    Args:
        global_model (nn.Module): The global model to be evaluated.
        test_loader (DataLoader): DataLoader for the test dataset.

    Returns:
        float: The accuracy of the model on the test dataset.
        float: The loss of the model on the test dataset.
    """
    test_accuracy, loss = evaluate(global_model, test_loader, criterion, DEVICE)
    return test_accuracy, loss

# DataLoading Process

We first need to import the file that contains the dataset we want to load for training and for testing porpouse.

If you are using colab we suggest you to change the following code block with:

from google.colab import files

uploaded2 = files.upload()

Please upload the training dataset provided by LEAF here.

In [4]:
root = tk.Tk()
#root.withdraw()

file_path = filedialog.askopenfilename(filetypes=[("JSON files", "*.json")])

if file_path:
    with open(file_path, 'r') as file:
        data = json.load(file)
            
root.destroy()

Please upload the test dataset provided by LEAF.

In [5]:
root = tk.Tk()
#root.withdraw()

file_path = filedialog.askopenfilename(filetypes=[("JSON files", "*.json")])

if file_path:
    with open(file_path, 'r') as file:
        test_data = json.load(file)
            
root.destroy()

In [6]:
num_clients = len(data['users'])
print("Number of clients:", num_clients)
NUM_CLIENTS = num_clients

Number of clients: 100


In [7]:
users = data['users']
num_samples = data['num_samples']
user_data = data['user_data']

In [8]:
all_texts = ''.join([''.join(seq) for user in users for seq in user_data[user]['x']])
chars = sorted(set(all_texts))
char_to_idx = {ch: idx for idx, ch in enumerate(chars)}

# Add the padding character
char_to_idx['<pad>'] = len(char_to_idx)
idx_to_char = {idx: ch for ch, idx in char_to_idx.items()}

## Convert data into indices

In [9]:
inputs = [[char_to_idx[char] for char in user_data[user]['x'][0]] for user in users]
targets = [[char_to_idx[char] for char in user_data[user]['y'][0]] for user in users]

## Creation of TensorDataset and DataLoader

In [10]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset


input_tensors = [torch.tensor(seq) for seq in inputs]
target_tensors = [torch.tensor([seq]) for seq in targets]

chars = sorted(set(all_texts))
char_to_idx = {ch: idx for idx, ch in enumerate(chars)}
char_to_idx['<pad>'] = len(char_to_idx)
idx_to_char = {idx: ch for ch, idx in char_to_idx.items()}

padded_inputs = pad_sequence(input_tensors, batch_first=True, padding_value=char_to_idx['<pad>'])

target_tensors = torch.cat(target_tensors, dim=0)

dataset = TensorDataset(padded_inputs, target_tensors)
batch_size = 4
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [11]:
def tensor_to_string(tensor, idx_to_char):
    """Converte un tensore di indici in una stringa di caratteri."""
    return ''.join(idx_to_char[idx.item()] for idx in tensor)

In [12]:
# Function to convert character values into indices
from torch.nn.utils.rnn import pad_sequence
from torch.nn.utils.rnn import pad_sequence
def char_to_tensor(characters):
    indices = [char_to_idx.get(char, char_to_idx['<pad>']) for char in characters] # Get the index for the character. If not found, use the index for padding.
    return torch.tensor(indices, dtype=torch.long)

# Prepare the training data_loader
# Prepara i dati di test
input_tensors = []
target_tensors = []
for user in data['users']:
    for entry, target in zip(data['user_data'][user]['x'], data['user_data'][user]['y']):
        input_tensors.append(char_to_tensor(entry))  # Use the full sequence of x
        target_tensors.append(char_to_tensor(target))  # Directly use the corresponding y as target

# Padding e creazione di DataLoader
padded_inputs = pad_sequence(input_tensors, batch_first=True, padding_value=char_to_idx['<pad>'])
targets = torch.cat(target_tensors)
dataset = TensorDataset(padded_inputs, targets)
for elem1, elem2 in dataset:
  elem2 = elem2.unsqueeze(0)

data_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)

In [13]:
# Prepare the test loader:
# Prepare the training data_loader

input_tensors = []
target_tensors = []
for user in test_data['users']:
    for entry, target in zip(test_data['user_data'][user]['x'], test_data['user_data'][user]['y']):
        input_tensors.append(char_to_tensor(entry))  # Use the full sequence of x
        target_tensors.append(char_to_tensor(target))  # Directly use the corresponding y as target

# Padding e creazione di DataLoader
padded_inputs = pad_sequence(input_tensors, batch_first=True, padding_value=char_to_idx['<pad>'])
targets = torch.cat(target_tensors)
dataset = TensorDataset(padded_inputs, targets)
for elem1, elem2 in dataset:
  elem2 = elem2.unsqueeze(0)

test_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)

## Model definition

In [14]:
global_model = CharLSTM(vocab_size=len(char_to_idx))
criterion = nn.CrossEntropyLoss()

# Evolutionary algorithm

In [15]:
import random
from copy import deepcopy
import os

import torch
import torch.nn as nn

#constants
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
#CRITERION = nn.NLLLoss()
#MOMENTUM = 0.9
#BATCHSIZE = 64

def tournament_selection_weakest(population, tau=2, p_diver=0.05):
    """
    Perform tournament selection to choose parents.
    Randomly select tau individuals and choose the weakest one.
    Fitness hole to introduce a 5% probability of choosing the fittest individual.


    :param population: List of Individuals.
    :param tau: Number of individuals to select.
    :param p_diver: Probability of choosing the worst individual in the tournament, done for the fitness hole.
    :return: Selected Individual.
    """
    participants = random.sample(population, tau)
    if random.random() < p_diver:
        winner = max(participants, key=lambda ind: ind.fitness)
    else:
      winner = min(participants, key=lambda ind: ind.fitness)
    return deepcopy(winner)

def tournament_selection_fittest(population, tau=2, p_diver=0.05):
    """
    Perform tournament selection to choose parents.
    Randomly select tau individuals and choose the best one.
    Fitness hole to introduce a 5% probability of choosing the weakest individual.


    :param population: List of Individuals.
    :param tau: Number of individuals to select.
    :param p_diver: Probability of choosing the worst individual in the tournament, done for the fitness hole.
    :return: Selected Individual.
    """
    participants = random.sample(population, tau)
    if random.random() < p_diver:
        winner = min(participants, key=lambda ind: ind.fitness)
    else:
      winner = max(participants, key=lambda ind: ind.fitness)
    return deepcopy(winner)


def client_size(individual, client_sizes):
    """
    Computes the number of total samples for individual
    """
    val = 0
    for client in individual.genome:
        val += client_sizes[client]
    return val


def EA_algorithm(generations, population_size, num_clients, num_classes, crossover_probability, dataset, lr, wd, criterion, char_to_idx, total_clients):
    """
    Perform the Evolutionary Algorithm (EA) to optimize the selection of clients.
    The EA consists of the following steps:
    1. Initialization: Create a population of individuals.
    2. Evaluation: Compute the fitness of each individual.
    3. Selection: Choose parents based on their fitness.
    4. Offspring to create the new population (generational model).
    6. Repeat from step 2 maximum iterations.

    :param generations: Number of generations to run the algorithm.
    :param population_size: Number of individuals in the population.
    :param num_clients: clients selected by each individual.
    :param num_classes: Number of classes for each client (iid or non-iid).
    :param crossover_probability: Probability of crossover for each individual.
    :param dataset: The dataset to be used for training.
    :param lr: The learning rate to be used for training.
    :param wd: The weight decay to be used for training.
    :param criterion: The loss function to use.
    :param char_to_idx: to switch between character and integer encoding of them.
    :param total_clients: total number of clients in the loaded database.


    :return global_model: The global model obtained after the EA.
    :return training_accuracies: The training loss of the global model at each generation.
    :return training_losses: The training accuracy of the global model at each generation.
    :return client_selection_count: The number of times each client was selected in the population.
    """


    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []
    # mantain memory of the number of times each client have been selected:
    client_selection_count = [0]*total_clients
    #print("num clients:,", total_clients)
    best_model_state = None
    best_train_loss = float('inf')


    # Initialize the population
    # Shuffle clients before assigning them
    all_clients = list(range(total_clients))
    random.shuffle(all_clients)

    #No individual, at the beginning, will select a client twice
    population = [
        Individual(genome=all_clients[i * num_clients:(i + 1) * num_clients], total_clients=total_clients)
        for i in range(population_size)
    ]
    #population = [Individual(genome=random.sample(range(100), k=num_clients)) for _ in range(population_size)]
    model = CharLSTM(vocab_size=len(char_to_idx))

    server = Server(model,DEVICE, char_to_idx)

    shards = server.sharding(dataset)
    client_sizes = [len(shard) for shard in shards]

    for gen in range(generations):
    #for gen in range(generations):
        # For each of them apply the fed_avg algorithm:
        param_list = []
        averages_acc = []
        average_loss = []
        for individual in population:
            #Update the client selection count
            for client in individual.genome:
                client_selection_count[client] += 1

            resulting_model, acc_res, loss_res = server.train_federated(criterion, lr, MOMENTUM, BATCH_SIZE, wd, individual, shards)
            param_list.append(resulting_model)
            averages_acc.append(acc_res)
            average_loss.append(loss_res)


        #Here we should average all the models to obtain the global model
        averaged_model,  train_loss, train_accuracy = server.fedavg_aggregate(param_list, [client_size(i, client_sizes) for i in population], average_loss, averages_acc)

        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)

        # Update the model with the result of the average:
        model.load_state_dict(averaged_model)
        #Just to be sure:
        server.global_model.load_state_dict(averaged_model)

        if train_loss < best_train_loss:
            best_train_loss = train_loss
            best_model_state = deepcopy(model.state_dict())

        offspring = []
        #Offspring-> offspring size is the same as population size
        elite = sorted(population, key=lambda ind: ind.fitness, reverse=True)[0]
        offspring.append(elite) #Keep the best individual
        for j in range(population_size-1):
            # Crossover
            if random.random() < crossover_probability:
                parent1 = tournament_selection_fittest(population)
                parent2 = tournament_selection_fittest(population)
                offspring.append(Individual.crossover(parent1, parent2))
            else:
                #Mutation
                parent = tournament_selection_weakest(population)
                parent.mutation()
                offspring.append(parent)

        # Replace the population with the new offspring
        population = offspring

    model.load_state_dict(best_model_state)
    return model, train_accuracies, train_losses, client_selection_count


In [16]:
# Parameters
lr = 1.0
wd = 0.0001
generations = 200
population_size = 10
num_clients = 2
num_classes = 100
crossover_probability = 0.5

In [17]:
print(BATCH_SIZE)

100


In [ ]:
#Best lr and wd found for iid federated baseline: lr=0.1, wd=0.001
global_model = CharLSTM(vocab_size=len(char_to_idx))
global_model,train_accuracies,train_losses,client_selection_count=EA_algorithm(generations=generations,population_size=population_size,num_clients=num_clients,num_classes = num_classes, crossover_probability = crossover_probability, dataset= data, lr =lr , wd = wd, criterion = criterion, char_to_idx=char_to_idx, total_clients=NUM_CLIENTS)
test_accuracy, test_loss = test(global_model, test_loader, criterion, DEVICE)
print("Test accuracy: ",test_accuracy)
#plot_client_selection(client_selection_count,"EA_iid_client_selection.png")
#save_data(global_model,val_accuracies,val_losses,train_accuracies,train_losses,client_selection_count,"EA_iid.pth")